In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
Tadv = 2.6

In [3]:
path = './'

f  = xr.open_dataset(path + 'astex_2010_v3a_SCAMIOP.nc')

In [4]:
f

<xarray.Dataset>
Dimensions:  (lat: 1, lev: 34, lon: 1, tsec: 41)
Coordinates:
  * lat      (lat) float32 34.0
  * lon      (lon) float32 -25.0
  * lev      (lev) float32 100.0 200.0 300.0 ... 95140.25 99185.59 102900.0
  * tsec     (tsec) int32 0 3600 7200 10800 ... 133200 136800 140400 144000
Data variables:
    calday   (tsec) float32 ...
    year     (tsec) int32 ...
    month    (tsec) int32 ...
    day      (tsec) int32 ...
    hour     (tsec) float32 ...
    nbdate   int32 ...
    bdate    int32 ...
    phis     (lat, lon) float32 ...
    Ps       (tsec, lat, lon) float32 ...
    Tg       (tsec, lat, lon) float32 ...
    T        (tsec, lev, lat, lon) float32 ...
    q        (tsec, lev, lat, lon) float32 ...
    u        (tsec, lev, lat, lon) float32 ...
    ug       (tsec, lev, lat, lon) float32 ...
    v        (tsec, lev, lat, lon) float32 ...
    vg       (tsec, lev, lat, lon) float32 ...
    divT     (tsec, lev, lat, lon) float32 ...
    divq     (tsec, lev, lat, lon) floa

In [5]:
# Create dictionaries for holding postprocessed variables
data_vars = {}
coords = {}
attrs = {}

# Save required coordinates
coords['lat'] = f['lat']
coords['lon'] = f['lon']
coords['lev'] = f['lev']
coords['tsec'] = f['tsec']

#get array size
ntsec = f['tsec'].size
nlev = f['lev'].size

In [20]:
#Extend time dimension
ntsec_out = 91
if ntsec_out - ntsec <= ntsec:
    f_out = xr.concat([f, f.isel(tsec = slice(0, ntsec_out - ntsec))], dim="tsec")
else:
    f_out = xr.concat([f, f.isel(tsec = slice(0, ntsec_out - ntsec)), 
                      f.isel(tsec = slice(0, ntsec_out - 2*ntsec))], dim="tsec")

In [21]:
# Modify those variables that are time-dependent 
# (initial variables should not change typically)

#time related variables
f_out.coords['tsec'] = np.arange(ntsec_out)*3600.
f_out['calday'] = xr.DataArray(
    f_out['calday'].values[0] + np.arange(ntsec_out)/24., 
    coords={'tsec': f_out['tsec'].values},
             dims=['tsec'])

#forcing variables that are time-varying
f_out['omega'] = f_out['omega'].isel(
    tsec = 0).expand_dims(
    tsec = f_out['tsec'].size).assign_coords(
    tsec=f_out['tsec'].values)

f_out['ug'] = f_out['ug'].isel(
    tsec = 0).expand_dims(
    tsec = f_out['tsec'].size).assign_coords(
    tsec=f_out['tsec'].values)

f_out['vg'] = f_out['vg'].isel(
    tsec = 0).expand_dims(
    tsec = f_out['tsec'].size).assign_coords(
    tsec=f_out['tsec'].values)

# Fit SST
coef_lin = np.polyfit(f['tsec'].values, f['Tg'].values[:,0,0], 1)
lfit = np.poly1d(coef_lin)
SST = lfit[0] + (-Tadv/(24.*3600.))*f_out['tsec'].values

f_out['Tg'] = xr.DataArray(
    SST.reshape(f_out['tsec'].size, f_out['lat'].size, f_out['lon'].size), 
    coords={'tsec': f_out['tsec'].values, 'lat': f_out['lat'].values, 'lon': f_out['lon'].values},
             dims=['tsec','lat','lon'])

In [22]:
# Drop some variables 
f_out.drop(['year', 'month', 'day', 'hour'])

<xarray.Dataset>
Dimensions:  (lat: 1, lev: 34, lon: 1, tsec: 91)
Coordinates:
  * lev      (lev) float32 100.0 200.0 300.0 ... 95140.25 99185.59 102900.0
  * lon      (lon) float32 -25.0
  * lat      (lat) float32 34.0
  * tsec     (tsec) float64 0.0 3.6e+03 7.2e+03 ... 3.168e+05 3.204e+05 3.24e+05
Data variables:
    calday   (tsec) float64 165.0 165.0 165.1 165.1 ... 168.6 168.7 168.7 168.8
    nbdate   (tsec) int32 920613 920613 920613 920613 ... 920613 920613 920613
    bdate    (tsec) int32 920613 920613 920613 920613 ... 920613 920613 920613
    phis     (tsec, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    Ps       (tsec, lat, lon) float32 102900.0 102900.0 ... 102900.0 102900.0
    Tg       (tsec, lat, lon) float64 291.1 291.0 290.9 ... 281.6 281.5 281.4
    T        (tsec, lev, lat, lon) float32 268.8749 267.1764 ... 290.3636
    q        (tsec, lev, lat, lon) float32 3.921989e-06 3.824483e-06 ... 0.0102
    u        (tsec, lev, lat, lon) float32 -2.0 -2.0 -2.0

In [23]:
#Drop (as array) and Add (as scalar) the nbdate and bdate
f_out['nbdate'] = f['nbdate']
f_out['bdate'] = f['bdate']

In [24]:
from datetime import date

f_out.attrs["author"] = "Youtong Zheng, zhengyoutong@gmail.com"
f_out.attrs["institution"] = "University of Maryland"
f_out.attrs["date"] = str(date.today())
f_out.attrs["comment"] = "Modified from forcing file for GCSS BLCWG Astex transition intercompariso..."

In [26]:
f_out

<xarray.Dataset>
Dimensions:  (lat: 1, lev: 34, lon: 1, tsec: 91)
Coordinates:
  * lev      (lev) float32 100.0 200.0 300.0 ... 95140.25 99185.59 102900.0
  * lon      (lon) float32 -25.0
  * lat      (lat) float32 34.0
  * tsec     (tsec) float64 0.0 3.6e+03 7.2e+03 ... 3.168e+05 3.204e+05 3.24e+05
Data variables:
    calday   (tsec) float64 165.0 165.0 165.1 165.1 ... 168.6 168.7 168.7 168.8
    year     (tsec) int32 1992 1992 1992 1992 1992 ... 1992 1992 1992 1992 1992
    month    (tsec) int32 6 6 6 6 6 6 6 6 6 6 6 6 6 ... 6 6 6 6 6 6 6 6 6 6 6 6
    day      (tsec) int32 13 13 13 13 13 13 13 13 13 ... 13 13 13 13 13 13 13 13
    hour     (tsec) float32 0.0 1.0 2.0 3.0 4.0 5.0 ... 3.0 4.0 5.0 6.0 7.0 8.0
    nbdate   int32 ...
    bdate    int32 ...
    phis     (tsec, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    Ps       (tsec, lat, lon) float32 102900.0 102900.0 ... 102900.0 102900.0
    Tg       (tsec, lat, lon) float64 291.1 291.0 290.9 ... 281.6 281.5 281.4
  

In [27]:
f_out.to_netcdf(
    path + 'astex_input_Tadv_' + str(Tadv) + '.nc')